<a href="https://colab.research.google.com/github/alejandromanas/GoogleColab_public/blob/master/Bookings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hotel Bookings


Libraries

In [0]:
from google.colab import drive
import pandas as pd
import numpy as np


### Loading Files from Drive

In [2]:
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
h1 = pd.read_csv("/content/drive/My Drive/colab files/bookings/H1.csv", delimiter = ',')
h2 = pd.read_csv("/content/drive/My Drive/colab files/bookings//H2.csv", delimiter = ',')

### Preprocessing

In [0]:
h1['hotel_type']='resort'
h2['hotel_type']='urban'
df=pd.concat([h1,h2], axis=0)

# Filling nan
df['Children'] = df.Children.replace(np.nan, 0)
df['Children'] = df.Children.astype('int')

#Convert to datetime
df['ReservationStatusDate']=pd.to_datetime(df.ReservationStatusDate)
df['ArrivalDateMonthNum']=df.ArrivalDateMonth.replace({ 'January' : 1, 'February' : 2, 'March' : 3, 'April' :4, 'May': 5, 'June' : 6, 'July':7, 'August' : 8, 'September' : 9, 'October' : 10, 'November': 11, 'December' : 12 })

# vtreat
X_vtreat=pd.DataFrame()
Y_vtreat=pd.DataFrame()
# X_test=df_model.loc[:, df_model.columns != 'IsCanceled'].copy()
X_vtreat=df_all[['Company','Agent']].copy()
Y_vtreat=df_all['IsCanceled'].copy()
transform = vtreat.BinomialOutcomeTreatment(
outcome_name='Y_vtreat',    # outcome variable
outcome_target=True,  # outcome of interest
cols_to_copy=['X_vtreat'],   # columns to "carry along" but not treat as input variables
   )

# plan = vtreat.BinomialOutcomeTreatment(outcome_target=True)
d_prepared = transform.fit_transform(X_vtreat, Y_vtreat)
# X_vtreat.shape 
# d_prepared.shape 
# list(d_prepared.columns)
df_all=pd.concat([df_all,d_prepared], axis=1)
# list(df_all.columns)